### Scoring Algorithm

In [ ]:
import pandas as pd

try:
    ### Import applicant data
    df = pd.read_csv('applicants.csv')

    ### Number of volunteers needed to select
    N = 5

    #Clean column names
    df.columns = df.columns.str.strip()

    #Copy for scoring
    df_score = df.copy()

    #Transform binary columns for scoring
    df_score['Past Volunteer (score)'] = df_score['Past Volunteer'].map({'Yes': 2, 'No': 0})
    df_score['Past Camper (score)'] = df_score['Past Camper'].map({'Yes': 1, 'No': 0})

    #Add count columns (keep raw columns intact)
    df_score['Course Count'] = df_score['Courses'].apply(lambda x: len(str(x).split(';')))
    df_score['Technical Skill Count'] = df_score['Technical Skills'].apply(lambda x: len(str(x).split(';')))

    #Map experience columns
    experience_map = {'0 hrs': 0, '1-9 hrs': 1, '10-20 hrs': 2, '20+ hrs': 3}
    df_score['Youth Experience (score)'] = df_score['Youth Experience'].map(experience_map)
    df_score['Leadership Experience (score)'] = df_score['Leadership Experience'].map(experience_map)

    #Calculate score based on counts and scoring columns
    df_score['Score'] = (
        df_score['Past Volunteer (score)'] +
        df_score['Past Camper (score)'] +
        df_score['Course Count'] +
        df_score['Technical Skill Count'] +
        df_score['Youth Experience (score)'] +
        df_score['Leadership Experience (score)']
    )

    #Sort by score and assign rank
    df_score = df_score.sort_values(by='Score', ascending=False).reset_index(drop=True)
    df_score.insert(0, 'Rank', df_score.index + 1)

    #Top N scorers
    df_score_top = df_score.head(N)

    #Save outputs
    df_score_top.to_csv("transformed_with_score.csv", index=False)
except:
    print('Something went wrong. Please check the input file and try again.')


### Scheduling Optimization

In [ ]:
import gurobipy as gp
from gurobipy import GRB

raw_schedule = pd.read_csv('raw_schedule.csv')
